In [ ]:
from sensible_raw.loaders import loader
#import geoplotlib as gpl
import numpy as np
from __future__ import division
import mpmath as mp
import json
import pandas as pd
from mpl_toolkits.basemap import Basemap

from geopy.distance import great_circle
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import calmap

plt.rcParams['figure.figsize'] = (10, 4)

In [ ]:
#Censored
def load_date(start_date,end_date):
    #creating a variable for the API call
            

In [ ]:
#Censored
#Input:  A  list with the months we want to look at
#Output:  A pandas dataframe with all the data
def load_data_all_months(months,data_type):
    #creating a pandas dataframe from an API call to the database

In [ ]:
#Making a sequence for each user
#Input: dataframe
#Output: A list of sequences for each user
def make_seq(dataframe, data_type):
    seq = {}
    if data_type == 'stop_locations':
        for i in set(dataframe['user']):
            seq[i]=[dataframe.loc[dataframe['user'].isin([i])]['label']][0].tolist()
    elif data_type == 'cell':
        for i in set(dataframe['user']):
            cid=[dataframe.loc[dataframe['user'].isin([i])]['cid']][0].tolist()
            lac=[dataframe.loc[dataframe['user'].isin([i])]['lac']][0].tolist()
            seq[i]=[float(str(int(x[0]))+'.'+str(int(x[1]))) for x in zip(cid,lac)]
    return seq

In [ ]:
#Editing our sequence. Making AAABBBCCCDD = ABCD and also removing all users who has less than a certain number N places
#Input:
#      seq: A list constaining lists of each user
#        N: An integer for the minimum of different locations visited after
def location_seq(seq,N=0):
    for i in seq.keys():
        j=1
        while j<len(seq[i]):
            if seq[i][j]==seq[i][j-1]:
                del seq[i][j]
            else:
                j+=1
    for i in seq.keys():
        if len(seq[i])<=N:
            del seq[i]
    return seq

In [ ]:
# State entropy (only num of states)
# S^{rand} = log2(N), N = 'number of states'
def func_S_rand(seq):
    return np.log2(len(np.unique(seq)))

In [ ]:
# Distribution entropy (uncorrelaed)
# S^{unc} = - sum_{i=1..n}(p_i * log2(p_i))
# Where p_i is the probability for state i.
# The sum of all probabilitys is 1: sum_{i=1..n}(p_i) = 1
def func_S_unc(seq):
    
    n = float(len(seq))
    states = np.unique(seq)
    N_states = len(np.unique(states))
    
    p = np.zeros(N_states)
    for i in range(N_states):
        p[i] = seq.count(states[i])/n
    
    return sum(-p * np.log2(p))

In [ ]:
# Lambda function for True entropy:
# Taken from Matlab function!
def func_Lambda(seq, i):
    n = len(seq)
    
    # Insert terminal symbol
    seq = seq + [min(seq)-1]
    
    x = 1
    mps = [idx for (idx, val) in enumerate(seq[:i]) if val == seq[i]]
    while (mps and x <= n-i):
        if mps[-1] + x >= i:
            del mps[-1]
            
        mps = [idx for idx in mps if seq[idx+x] == seq[i+x]]
        x += 1
        
    return x

In [ ]:
# True entropy: Distribution entropy (correlaed), looks at correlations in the sequens
# S = - sum_{T'_i in T_i}(P(T'_i) * log2(P(T'_i)))
def func_S(seq):
    
    n = len(seq)
    L = np.zeros(n)
    
    for i in range(n):
        # Lambda function call
        L[i] = func_Lambda(seq,i)
    
    # Function from Gavin Smith (LoPpercom)
    # S = np.power(sum([L[i] / np.log2(i+1) for i in range(1,n)] ) * (1.0/n),-1)
    # Function from Morten Proschowsky (Matlab)
    S = n/sum(L) * np.log2(n)
    return S

In [ ]:
# Predictability solving: 
# solve(0 = [-x*log2(x)-(1-x)*log2(1-x)+(1-x)*log2(N-1)] - S
# Returns value between 0 and 1.
def func_Pred(S_score,N_states):
    
    if N_states <= 1:
        return 1
    
    # Convex function for the predictibility bound
    func = lambda x, S, N: (-(x*mp.log(x,2)+(1-x)*mp.log(1-x,2))+(1-x)*mp.log(N-1,2))-S
    func2 = lambda x: func(x,S_score,N_states)
    # Solve function f(x) = 0
    res = mp.findroot(func2,0.95).real
    
    return float(res)

In [ ]:
# Returns S_rand, S_unc, S, U_rand, U_unc and U_max.
# Set print_res to True to print results
def Compute_all(seq, print_res = False):
    N_states = len(np.unique(seq))
    S_rand = func_S_rand(seq)
    S_unc  = func_S_unc(seq)
    S = func_S(seq)
    U_rand = func_Pred(S_rand,N_states)
    U_unc  = func_Pred(S_unc,N_states)
    U_max  = func_Pred(S,N_states)
    
    if print_res:
        print 'S_rand: ' + str(S_rand)
        print 'S_unc:  ' + str(S_unc)
        print 'S:      ' + str(S)
        print 'U_rand: ' + str(U_rand)
        print 'U_unc:  ' + str(U_unc)
        print 'U_max:  ' + str(U_max)
        
    return (S_rand, S_unc, S, U_rand, U_unc, U_max)

In [ ]:
def entropy_convergence(seq, lists):
    seq_entropy = {}
    for i in seq.keys():
        seq_entropy[i]=[]
    for i in lists:
        seq_temp = location_seq(seq,i)
        for key in seq_temp.keys():
            seq_entropy[key].append(func_S(seq[key][:i]))
    return seq_entropy

In [ ]:
def calmap_plot(df,time_type):
    Dates = pd.to_datetime(df['timestamp'],unit = time_type)
    df_heat = pd.Series([1 for i in range(df.shape[0])],index = Dates)
    f,ax = calmap.calendarplot(df_heat,fillcolor=(0,0,0.5),cmap='gist_heat_r')

# Checking data quality.

In [ ]:
# loading data
df_loc = load_data_all_months(load_date([3,2014],[2,2015]),'stop_locations')

In [ ]:
# Checking how many weeks are complete for each user
complete_data = {}
for user in set(df_loc['user']):
    complete_data[user]=[]
    df_loc_temp = df_loc[df_loc['user']==user]
    time = df_loc_temp.iloc[0]['arrival']  #start time
    time_day = 86400
    while time < df_loc_temp.iloc[-1]['departure']:
        count = 0
        for day in range(7):
            if df_loc_temp[(df_loc_temp['arrival']>=time) & (df_loc_temp['arrival']<time+time_day)].size!=0:
                count+=1
            time +=time_day
        if count>=5:
            complete_data[user].append(1)
        else:
            complete_data[user].append(0)            

In [ ]:
# Fetching the best users according to our requirements
best_users = []
for key in complete_data.keys():
    if len(complete_data[key])>50 and sum(complete_data[key])/len(complete_data[key])>0.85:
        best_users.append([key,sum(complete_data[key])/len(complete_data[key])])

In [ ]:
#Makes the sequence location based and calculates the lengths of the user sequences
data_points = []
seq_full = make_seq(df_loc,'stop_locations')
seq_full = location_seq(seq_full)
for user_seq in seq_full.values():
    data_points.append(len(user_seq))

In [ ]:
plt.figure(figsize=(7,4.5))
plt.hist(data_points,range(0,max(data_points)+100,100))
plt.xlabel("Sequence length")
plt.ylabel("Amount of users")

In [ ]:
# Getting the best users of the requirements earlier
seq_best = {}
for i in best_users:
    seq_best[i[0]]=seq_full[i[0]]
seq_best = location_seq(seq_best)

In [ ]:
data_points_compl = []
for user_seq in seq_best.values():
    data_points_compl.append(len(user_seq))

In [ ]:
plt.figure(figsize=(7,4.5))
plt.hist(data_points_compl,range(0,max(data_points_compl)+100,100))
plt.xlabel("Sequence length")
plt.ylabel("Amount of users")

# Entropy

In [ ]:
# Calculating the entropy and predictability for each user for the full data
S_rand_all, S_unc_all, S_all, U_rand_all, U_unc_all, U_max_all=[],[],[],[],[],[]
for seq in seq_full.values():
    S_rand, S_unc, S, U_rand, U_unc, U_max = Compute_all(seq)
    S_rand_all.append(S_rand)
    S_unc_all.append(S_unc)
    S_all.append(S)
    U_rand_all.append(U_rand)
    U_unc_all.append(U_unc)
    U_max_all.append(U_max)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(231)
plt.hist(S_rand_all,20)
plt.title('S_rand')
plt.ylabel('Amount of users')
plt.xlabel("Entropy")

plt.subplot(232)
plt.hist(S_unc_all,20)
plt.title('S_unc')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(233)
plt.hist(S_all,20)
plt.title('S_temp')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(234)
plt.hist(U_rand_all,20,range=(0,1))
plt.title('U_rand')
plt.ylabel('Amount of users')
plt.xlabel("Predictability")

plt.subplot(235)
plt.hist(U_unc_all,20,range=(0,1))
plt.title('U_unc')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.subplot(236)
plt.hist(U_max_all,20,range=(0,1))
plt.title('U_temp')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.show


In [ ]:
plt.figure(figsize=(15,2.5))

plt.subplot(131)
plt.hist(S_rand_all,np.arange(0,10,10/20))
plt.title('S_rand')
plt.ylabel('Amount of users')
plt.xlabel("Entropy")

plt.subplot(132)
plt.hist(S_unc_all,np.arange(0,8,8/20))
plt.title('S_unc')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(133)
plt.hist(S_all,np.arange(0,6,6/20))
plt.title('S_temp')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

In [ ]:
#Calculating the entropy and predictability for the best users
S_rand_all, S_unc_all, S_all, U_rand_all, U_unc_all, U_max_all=[],[],[],[],[],[]
for seq in seq_best.values():
    S_rand, S_unc, S, U_rand, U_unc, U_max = Compute_all(seq)
    S_rand_all.append(S_rand)
    S_unc_all.append(S_unc)
    S_all.append(S)
    U_rand_all.append(U_rand)
    U_unc_all.append(U_unc)
    U_max_all.append(U_max)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(231)
plt.hist(S_rand_all,20)
plt.title('S_rand')
plt.ylabel('Amount of users')
plt.xlabel("Entropy")

plt.subplot(232)
plt.hist(S_unc_all,20)
plt.title('S_unc')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(233)
plt.hist(S_all,20)
plt.title('S_temp')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(234)
plt.hist(U_rand_all,20,range=(0,1))
plt.title('U_rand')
plt.ylabel('Amount of users')
plt.xlabel("Predictability")

plt.subplot(235)
plt.hist(U_unc_all,20,range=(0,1))
plt.title('U_unc')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.subplot(236)
plt.hist(U_max_all,20,range=(0,1))
plt.title('U_temp')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.show

# Testing entropy convergence.

In [ ]:
# Loading the data
df_loc = load_data_all_months(load_date([3,2014],[2,2015]),'stop_locations')

In [ ]:
# Make it location based
seq_full  = make_seq(df_loc,'stop_locations')

In [ ]:
#Computing the entropy convergence
Curves = []
Curves_index = []
for i in range(len(seq_best.keys())):
    index = seq_best.keys()[i]
    a = entropy_convergence({index:seq_best[index]},range(10,3000,10))[index]
    Curves.append(a) 
    Curves_index.append(index)
    #plt.plot(range(10,3000,10)[:len(a)],a,label = 'User '+ str(index))
    plt.plot(range(10,3000,10)[:len(a)],a)
    plt.legend()
    plt.xlabel("Sequence length")
    plt.ylabel("Entropy")

In [ ]:
# And the slope of the convergence
for i in range(len(Curves)):
    curve = [abs(Curves[i][j]-Curves[i][j+5]) for j in range(len(Curves[i])-5)]
    plt.plot(range(10,3000,10)[:len(curve)],curve)
    plt.xlabel("Sequence length")
    plt.ylabel("Slope")
    

In [ ]:
#Finding the best users according to our new requirement
count = 0
count_best_users = []
for i in range(len(seq_best.keys())):
    curve = [abs(Curves[i][j]-Curves[i][j+5]) for j in range(len(Curves[i])-5)]
    for j in range(len(curve)-10):
        if sorted(curve[j:j+10])[-3]<0.05:
            count+=1
            count_best_users.append(Curves_index[i])
            break
        

In [ ]:
#Making a dict of the new best users
seq_best_final = {}
for user in count_best_users:
    seq_best_final[user] = seq_best[user]

In [ ]:
# Calculating the entropy and predictability for the best users
S_rand_all, S_unc_all, S_all, U_rand_all, U_unc_all, U_max_all=[],[],[],[],[],[]
for seq in seq_best_final.values():
    S_rand, S_unc, S, U_rand, U_unc, U_max = Compute_all(seq)
    S_rand_all.append(S_rand)
    S_unc_all.append(S_unc)
    S_all.append(S)
    U_rand_all.append(U_rand)
    U_unc_all.append(U_unc)
    U_max_all.append(U_max)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(231)
plt.hist(S_rand_all,20)
plt.title('S_rand')
plt.ylabel('Amount of users')
plt.xlabel("Entropy")

plt.subplot(232)
plt.hist(S_unc_all,20)
plt.title('S_unc')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(233)
plt.hist(S_all,20)
plt.title('S_temp')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(234)
plt.hist(U_rand_all,20,range=(0,1))
plt.title('U_rand')
plt.ylabel('Amount of users')
plt.xlabel("Predictability")

plt.subplot(235)
plt.hist(U_unc_all,20,range=(0,1))
plt.title('U_unc')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.subplot(236)
plt.hist(U_max_all,20,range=(0,1))
plt.title('U_temp')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.show

In [ ]:
plt.figure(figsize=(15,2.5))

plt.subplot(131)
plt.hist(S_rand_all,np.arange(0,10,10/20))
plt.title('S_rand')
plt.ylabel('Amount of users')
plt.xlabel("Entropy")

plt.subplot(132)
plt.hist(S_unc_all,np.arange(0,8,8/20))
plt.title('S_unc')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

plt.subplot(133)
plt.hist(S_all,np.arange(0,6,6/20))
plt.title('S_temp')
plt.xlabel("Entropy")
plt.ylabel('Amount of users')

In [ ]:
plt.figure(figsize=(15,2.5))

plt.subplot(131)
plt.hist(U_rand_all,20,range=(0,1))
plt.title('U_rand')
plt.ylabel('Amount of users')
plt.xlabel("Predictability")

plt.subplot(132)
plt.hist(U_unc_all,20,range=(0,1))
plt.title('U_unc')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.subplot(133)
plt.hist(U_max_all,20,range=(0,1))
plt.title('U_temp')
plt.xlabel("Predictability")
plt.ylabel('Amount of users')

plt.show

In [ ]:
# Making a pandas data frame to save our new best users
df_user_entropy = 0
df_user_entropy = pd.DataFrame(columns=('user','S_ran','S_unc','S_temp','Length','unique'))
index=0
for user in seq_best_final.keys():
    seq = seq_best[user]
    seq_len = len(seq)
    S_rand = func_S_rand(seq)
    S_unc  = func_S_unc(seq)
    S = func_S(seq)
    unique = len(np.unique(seq)) 
    df_user_entropy.loc[index]=[int(user),S_rand,S_unc,S,seq_len,unique]
    index+=1

In [ ]:
#Saving the best users
df_user_entropy.to_csv('best_users_stop_entropy.csv')

# Check if cellIDS under a lac are the same

In [ ]:
#Import the cell locations
dataframe_cells = pd.read_csv('cell_towers_DK_stor.csv')   #For mylnikov.org/archives/1059
cell_dict_DK = {}
for coloums, row in dataframe_cells.iterrows():
    net,area, cell, lon, lat = row[5],row[6],row[7],row[9],row[8]    #mylnikov
    if area in cell_dict_DK:
        if cell in cell_dict_DK[area]:
            cell_dict_DK[area][cell].append([lat,lon])
        else:
            cell_dict_DK[area][cell]=[[lat,lon]]
    else:
        cell_dict_DK[area]={cell:[[lat,lon]]}

In [ ]:

cell_dict_DK[23033]

# Heat maps

In [ ]:
df_loc_heat = load_data_all_months(load_date([3,2014],[2,2015]),'stop_locations')

In [ ]:
#Full data
calmap_plot(df_loc_heat,'ms')

In [ ]:
#Some of the best users
calmap_plot(df_loc[df_loc['user'].isin(seq_best_final.keys())],'ms')

# Quality of the raw data

In [ ]:
df__all = load_data_all_months(load_date([3,2014],[2,2015]),'location')

In [ ]:
#Number of points in a year
yearly_points = 31556926000/900000
fraction_of_best = []
fraction_user_best =[]
for user in seq_best.keys():
    fraction_of_best.append(df__all[df__all['user']==user].shape[0]/yearly_points)
    fraction_user_best.append(user)

In [ ]:
#Number of points in a year
yearly_points = 31556926000/900000
fraction_of_full = []
fraction_user_full =[]
for user in df__all['user'].unique():
    fraction_of_full.append(df__all[df__all['user']==user].shape[0]/yearly_points)
    fraction_user_full.append(user)

In [ ]:

plt.figure()

plt.subplot(121)
plt.hist(fraction_of_full,np.arange(0,1.1,0.05))
plt.title('All users')
plt.xlabel('Completness', fontsize=10)
plt.ylabel('Amount of users', fontsize=10)

plt.subplot(122)
plt.hist(fraction_of_best,np.arange(0,1.1,0.05))
plt.title('Best users')
plt.xlabel('Completness', fontsize=10)
plt.ylabel('Amount of users', fontsize=10)

In [ ]:
plt.figure(figsize=(5,4))
plt.hist(fraction_of_full,np.arange(0,1.1,0.05))
plt.title('All users')
plt.xlabel('Completness', fontsize=10)
plt.ylabel('Amount of users', fontsize=10)